Check connection to cluster is healthy

In [7]:
%status

{'status': 'healthy',
 'startTime': 'Sat May 20 04:26:30 UTC 2023',
 'dbEngineVersion': '1.2.0.2.R3',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.5.5'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000'}}

Test if the dataset is empty or not

In [48]:
%%oc
MATCH (n)
RETURN COUNT(n)

Delete all nodes and relationships

In [36]:
%%oc
MATCH (n) WITH n LIMIT 100000
DETACH DELETE n
RETURN COUNT(*)

Alternative for resetting database (for large instances)

Generate token to reset database

In [8]:
%db_reset --generate-token

{'status': '200 OK',
 'payload': {'token': '32c41bec-0a9d-0a31-8fd2-d53791f5467a'}}

Reset database using token

In [9]:
%db_reset --token '32c41bec-0a9d-0a31-8fd2-d53791f5467a'

Load graph data from S3 bucket

Source: s3://northwind-graph-data

Load ARN: arn:aws:iam::925004103030:role/NeptuneLoadFromS3

In [38]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

Check loading status (replace Load ID with current ID)

In [6]:
%load_status 38b999bc-c99c-48fd-b7e6-179673cf9d5a

{
  "status": "200 OK",
  "payload": {
    "feedCount": [
      {
        "LOAD_COMPLETED": 1
      }
    ],
    "overallStatus": {
      "fullUri": "s3://graph-import",
      "runNumber": 1,
      "retryNumber": 0,
      "status": "LOAD_COMPLETED",
      "totalTimeSpent": 2041,
      "startTime": 1683249579,
      "totalRecords": 21486375,
      "totalDuplicates": 0,
      "parsingErrors": 0,
      "datatypeMismatchErrors": 0,
      "insertErrors": 0
    }
  }
}


Execute aggreation query in denormalized graph

In [55]:
%%oc
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
WITH o.customerID AS orders, COUNT(*) AS amount
RETURN min(amount), max(amount), avg(amount)

Execute update on denormalized graph (min redundancy)

In [65]:
%%oc
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion) AND
o.customerID = 'SAVEA'
SET o.shipCountry = 'United States'

Execute update on denormalized graph (avg redundancy)

In [57]:
%%oc
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion) AND
o.customerID = 'SEVES'
SET o.shipCountry = 'United Kingdom'

Execute update on denormalized graph (max redundancy)

In [58]:
%%oc
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion) AND
o.customerID = 'SAVEA'
SET o.shipCountry = 'United States'

Normalize graph: Update customer nodes (rather than creating new nodes) for each equivalence class

In [66]:
%%oc
MATCH (o:Order),(c:Customer) WHERE
EXISTS(c.customerID) AND
EXISTS(o.customerID) AND
(o.customerID = c.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
SET c.shipCity = o.shipCity
REMOVE o.shipCity
SET c.shipName = o.shipName
REMOVE o.shipName
SET c.shipPostalCode = o.shipPostalCode
REMOVE o.shipPostalCode
SET c.shipCountry = o.shipCountry
REMOVE o.shipCountry
SET c.shipAddress = o.shipAddress
REMOVE o.shipAddress
SET c.shipRegion = o.shipRegion
REMOVE o.shipRegion

Normalize graph: No need to create new edges as in this instance no new nodes have been created

Remove properties related to equivalence class from original nodes

In [67]:
%%oc
MATCH (o:Order) REMOVE o.customerID

Execute aggreation query on normalized graph

In [81]:
%%oc
MATCH (c:Customer) WHERE
EXISTS(c.customerID) AND
EXISTS(c.shipCity) AND
EXISTS(c.shipName) AND
EXISTS(c.shipPostalCode) AND
EXISTS(c.shipCountry) AND
EXISTS(c.shipAddress) AND
EXISTS(c.shipRegion)
WITH SIZE((c)--()) AS amount
RETURN min(amount), max(amount), avg(amount)

Execute update on normalized graph (min redundancy)

In [77]:
%%oc
MATCH (c:Customer) WHERE 
EXISTS(c.customerID) AND
EXISTS(c.shipCity) AND
EXISTS(c.shipName) AND
EXISTS(c.shipPostalCode) AND
EXISTS(c.shipCountry) AND
EXISTS(c.shipAddress) AND
EXISTS(c.shipRegion) AND
c.customerID = 'CENTC'
SET c.shipCountry = 'Estados Unidos Mexicanos'

Execute update on normalized graph (avg redundancy)

In [71]:
%%oc
MATCH (c:Customer) WHERE 
EXISTS(c.customerID) AND
EXISTS(c.shipCity) AND
EXISTS(c.shipName) AND
EXISTS(c.shipPostalCode) AND
EXISTS(c.shipCountry) AND
EXISTS(c.shipAddress) AND
EXISTS(c.shipRegion) AND
c.customerID = 'SEVES'
SET c.shipCountry = 'United Kingdom'

Execute update on normalized graph (max redundancy)

In [72]:
%%oc
MATCH (c:Customer) WHERE 
EXISTS(c.customerID) AND
EXISTS(c.shipCity) AND
EXISTS(c.shipName) AND
EXISTS(c.shipPostalCode) AND
EXISTS(c.shipCountry) AND
EXISTS(c.shipAddress) AND
EXISTS(c.shipRegion) AND
c.customerID = 'SAVEA'
SET c.shipCountry = 'United States'